In [8]:
%load_ext autoreload 
%autoreload 2

In [9]:
import os

# os.chdir("..")
os.chdir("..")

In [10]:
import pandas as pd
import sqlite3
from pathlib import Path

from modelagem.utils.logs import logger
from modelagem.train import model_trainer

# Ler 500 colunas no pandas 
pd.set_option('display.max_columns', 500)

2025-04-16 13:21:33,203 | main | INFO | logs | <module> | Logger configurado com sucesso.


In [27]:
from modelagem.train.model_trainer import load_model, load_json
from modelagem.settings.config import Settings
config = Settings()

In [40]:
import os
import pickle
import json
from sklearn.metrics import accuracy_score, f1_score
from modelagem.utils.metrics import (
    get_precision, 
    get_multiclass_confusion_counts, 
    calc_recall, 
    calc_f1)
from sklearn.preprocessing import StandardScaler
import pandas as pd
from pathlib import Path

def calc_metrics_multclass(dict_confusion_counts, result_mapping):

    [dict_confusion_counts.get(key) for key in dict_confusion_counts.keys()]
    result_id_mapping = {result_mapping.get(k):k for k in result_mapping}

    list_metrics_multclass = []
    for key in dict_confusion_counts.keys():
        key_confusion = dict_confusion_counts.get(key)
        recall = calc_recall(tp=key_confusion.get('tp'),
                                            fn=key_confusion.get('fn'))

        f1 = calc_f1(precision=precision, recall=recall)
        
        # print(result_id_mapping.get(key), recall, f1)
        list_metrics_multclass.append({
                "result":result_id_mapping.get(key),
                "recall": recall*100,
                "f1": f1*100,
            })
    return list_metrics_multclass

path_models = Path("database/models")
results = []

if not path_models.exists():
    print("❌ Diretório 'database/models' não encontrado.")
else:
    for folders in os.listdir(path_models):
        path_files = path_models / folders
        list_files = os.listdir(path_files)

        if any(".pkl" in file for file in list_files):
            try:
                print(f"\n🧪 Testando modelo na pasta: {folders}")
                path_model = path_files / "logistic_regression_model.pkl"
                path_mapping_feature_order = path_files / "mapping" / "feature_order.json"
                path_mapping_result_mapping = path_files / "mapping" / "result_mapping.json"

                path_feature = path_files / "features" / "ft_df.csv"

                if not path_model.exists() or not path_feature.exists() or not path_mapping_feature_order.exists():
                    print("❌ Arquivos necessários não encontrados, pulando...")
                    continue

                # Carrega os dados
                df = pd.read_csv(path_feature)
                df_validate = df[df["season"] >= 2024]
                target_column = 'result_encoded'
                x_validate = df_validate.drop(columns=target_column)
                y_validate = df_validate[target_column]

                # Carrega o modelo
                with open(path_model, 'rb') as f:
                    model = pickle.load(f)

                # Carrega a ordem das features
                result_mapping = load_json(path_mapping_result_mapping)
                feature_order = load_json(path_mapping_feature_order)

                # Prepara os dados
                X_test = x_validate[feature_order]
                scaler = StandardScaler()
                X_test_scaled = scaler.fit_transform(X_test)

                # Avalia
                y_pred = model.predict(X_test_scaled)
                # acc = accuracy_score(y_validate, y_pred)
                # f1 = f1_score(y_validate, y_pred, average='weighted')

                precision = get_precision(y_pred=y_pred, 
                                          y_true=y_validate)
                
                dict_confusion_counts = get_multiclass_confusion_counts(y_pred=y_pred, 
                                          y_true=y_validate)
                

                recall = calc_recall(tp=dict_confusion_counts.get('tp'),
                                    fn=dict_confusion_counts.get('fn'))

                f1 = calc_f1(precision=precision, recall=recall)

                list_metrics_multclass = calc_metrics_multclass(dict_confusion_counts, result_mapping)

                print(list_metrics_multclass)

                print('---'*20)
                results.append({
                    "model": folders,
                    "accuracy": precision,
                    "f1_score": f1,
                    "recall":recall,
                })

            except Exception as e:
                print(f"⚠️ Erro ao testar o modelo {folders}: {e}")

    if results:
        df_results = pd.DataFrame(results)
        print("\n📊 Resultados:")
        print(df_results.sort_values(by="f1_score", ascending=False))
    else:
        print("⚠️ Nenhum modelo válido foi testado.")



🧪 Testando modelo na pasta: 20250416_131211_first
[{'result': 'A', 'recall': 38.0, 'f1': 40.0}, {'result': 'D', 'recall': 32.0, 'f1': 36.0}, {'result': 'H', 'recall': 49.0, 'f1': 45.0}]
------------------------------------------------------------

🧪 Testando modelo na pasta: 20250416_131321_basic
[{'result': 'A', 'recall': 2.0, 'f1': 4.0}, {'result': 'D', 'recall': 53.0, 'f1': 44.0}, {'result': 'H', 'recall': 49.0, 'f1': 43.0}]
------------------------------------------------------------

📊 Resultados:
                   model  accuracy  f1_score  recall
0  20250416_131211_first  0.417949       NaN     NaN
1  20250416_131321_basic  0.382051       NaN     NaN


c:\pyprojects\pred_soccer\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\pyprojects\pred_soccer\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [39]:
list_metrics_multclass

[{'result': 'A', 'recall': 2.0, 'f1': 4.0},
 {'result': 'D', 'recall': 53.0, 'f1': 44.0},
 {'result': 'H', 'recall': 49.0, 'f1': 43.0}]

In [ ]:
#                   model  accuracy  f1_score  \
# 0  20250416_131211_first  0.417949  0.424845   
# 1  20250416_131321_basic  0.382051  0.346784

In [25]:
path_model = list_path_models_ok[0]

path_model

'database/models\\20250416_113549_teste_inplementacao'

In [32]:
df = pd.read_csv(r"C:\pyprojects\pred_soccer\database\models\20250416_113609_teste_inplementacao\features\ft_df.csv")

df_validate = df[df["season"] >= 2024]

target_column = 'result_encoded'

x_validate = df_validate.drop(columns=target_column)
y_validate = df_validate[target_column]